In [ ]:
import pymongo
import faker
import json
from bson import ObjectId
import hashlib

In [ ]:
# Connect to the MongoDB with user and password
db = pymongo.MongoClient("mongodb://localhost:27017/")["simulator"]

In [ ]:
# Create a fake truck data generator
faker.Faker.seed(42)

fake = faker.Faker("es_ES")
trucks = db["trucks"]
#drop the collection
trucks.drop()
for i in range(20):
    truck = {
        "id": fake.license_plate_unified().replace(" ", ""),
        "capacity": fake.random_int(min=10, max=30)
    }
    trucks.insert_one(truck)

In [ ]:
def create_object_id(string):
    hashed = hashlib.sha1(string.encode()).hexdigest()
    object_id = ObjectId(hashed[:24])
    return object_id

In [ ]:
locations = db["locations"]
#drop the collection
locations.drop()
farmacias = json.load(open("../farmacias/farmacias.json", "r"))
#filter farmacias with no coordinates
farmacias = [farmacia for farmacia in farmacias if farmacia["coordinates"] is not None and farmacia["coordinates"]["latitude"] is not None and farmacia["coordinates"]["longitude"] is not None]
# eliminate farmacias with no name or duplicate names
farmacias = {farmacia["name"]: farmacia for farmacia in farmacias}.values()

#add _id
for farmacia in farmacias:
    farmacia["_id"] = create_object_id(farmacia["name"])

locations.insert_many(farmacias)

In [ ]:
# plot al the locations in a map
import folium

m = folium.Map(location=[40.416775, -3.703790], zoom_start=9)
for farmacia in farmacias:
    folium.Marker([farmacia["coordinates"]["latitude"], farmacia["coordinates"]["longitude"]], popup=farmacia["name"]).add_to(m)
m